Given the `metadata/countries.json` file created by `Prepare Sources`, fetch each of the referenced linked data resources.

We'll just RDFlib to parse the RDF and serialize it back out as one big file.  We could just use `Graph.parse()` to directly fetch and parse the source data, but to cache the response, we'll fetch the RDF and put it in the `in` folder if it's not there already.

In [ ]:
import json
import requests
from pathlib import Path
from rdflib import Graph, URIRef, Namespace, RDF, Literal, RDFS, term, XSD
from rdflib.namespace import SKOS

# monkey patch to avoid gYear "normalization" issue https://github.com/RDFLib/rdflib/issues/806
term._toPythonMapping.pop(XSD['gYear'])

FOI = Namespace('http://publishmydata.com/def/ontology/foi/')

countries_json = Path('metadata') / 'countries.json'

countries = json.load(open('metadata/countries.json'))
countriesGraph = Graph()
countriesGraph.bind('foi', FOI)
countriesCollection = URIRef('http://gss-data.org.uk/def/trade/countries#collection')
countriesGraph.add((countriesCollection, RDF.type, FOI.AreaCollection))
countriesGraph.add((countriesCollection, RDFS.label, Literal('Countries')))
countriesGraph.add((countriesCollection, FOI.singularDisplayName, Literal('Country')))

sourceFolder = Path('in')
sourceFolder.mkdir(exist_ok=True)

tgnFile = sourceFolder / 'tgn.ttl'
if not(tgnFile.exists() and tgnFile.is_file()):
    response = requests.get('http://vocab.getty.edu/tgn/', headers={'Accept': 'text/turtle'})
    with open(tgnFile, 'wb') as f:
        f.write(response.content)
countriesGraph.parse(open(tgnFile), format='turtle')

for country, tgn_list in list(countries.items()):
    if len(tgn_list) > 0:
        tgn_uri = tgn_list[0]
        tgn_id = tgn_uri[len('http://vocab.getty.edu/tgn/'):]
        sourceFile = sourceFolder / (tgn_id + '.ttl')
        if not(sourceFile.exists() and sourceFile.is_file()):
            response = requests.get(tgn_uri, headers={'Accept': 'text/turtle'})
            with open(sourceFile, 'wb') as f:
                f.write(response.content)
        countriesGraph.parse(open(sourceFile), format='turtle')
        tgn = URIRef(tgn_uri)
        countriesGraph.add((tgn, RDF.type, FOI.Feature))
        countriesGraph.add((tgn, FOI.memberOf, countriesCollection))
        for label in countriesGraph.objects(tgn, RDFS.label):
            countriesGraph.add((tgn, FOI.displayName, label))

destinationFolder = Path('out')
destinationFolder.mkdir(exist_ok=True)

countries_ttl = destinationFolder / 'countries.ttl'
with open(countries_ttl, 'wb') as o:
    countriesGraph.serialize(destination=o, format='turtle')